Project to examine trends in Cryptocurrencies

In [104]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import os
import json

# Import API key
from api_keys import nomics_key
from api_keys2 import alvan_key
from gold_api import gold_api
from bls_key import bls_key

Extract Cryptocurrency historical price data

In [3]:
# Create start and end dates for queries since the API goes from daily data to weekly data if you query more than a month
# and there is no parameter that allows you to change the interval
years = ['2013','2014','2015','2016','2017','2018','2019','2020','2021']
days_per_month = {'01':'31','02':'28','03':'31','04':'30','05':'31','06':'30','07':'31','08':'31','09':'30','10':'31','11':'30','12':'31'}
btc_end_dates = []
btc_start_dates = []
for year in years:
    for key, values in days_per_month.items():
        btc_end_dates.append(f'{year}-{key}-{values}')
        btc_start_dates.append(f'{year}-{key}-01')
btc_end_dates = btc_end_dates[1:97] 
btc_start_dates = btc_start_dates[1:97]
btc_intervals = dict(zip(btc_start_dates, btc_end_dates))

In [5]:
# Query for Bitcoin data from 2/1/2013 to 1/31/2021
url = 'https://api.nomics.com/v1'
btc_timestamps = []
btc_prices = []
for key, value in btc_intervals.items():
    query_url = f'{url}/currencies/sparkline?key={nomics_key}&ids=BTC&start={key}T00%3A00%3A00Z&end={value}T00%3A00%3A00Z&interval=1d'
    data = requests.get(query_url)
    day_price = data.json()
    btc_timestamps.extend(day_price[0]['timestamps'])
    btc_prices.extend(day_price[0]['prices'])

In [6]:
# Remove some of the date coding from the BTC timestamps
btc_timestamps[:] = [s.replace('00:00:00Z','') for s in btc_timestamps]

In [7]:
btc_timestamps[:] = [s.replace('T','') for s in btc_timestamps]

In [8]:
# Make lists of weekly BTC prices and dates for analysis with gold
crypto_weekly_dates = []
crypto_weekly_prices = []
c_index = 0
for c in range(417):
    crypto_weekly_dates.append(btc_timestamps[c_index])
    crypto_weekly_prices.append(btc_prices[c_index])
    c_index = (c+1)*7

In [9]:
#create a dataframe to hold the crypto date and price data
crypto_dict = {'Date':btc_timestamps, 'BTC Price':btc_prices}
crypto_df = pd.DataFrame(crypto_dict)
crypto_df.to_csv('Crypto_Prices.csv')

EXTRACT STOCK DATA

In [224]:
# Query for DJIA and NDAQ daily stock data
# tickers = ['DJI','NDAQ','SWTX']
# stock_url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=SWTX&datatype=json&apikey={alvan_key}'
# stock_pull = requests.get(stock_url)
# dji_json = stock_pull.json()
# dji_json

EXTRACT GOLD DATA

In [11]:
# Create weekly list of dates for query
gold_dates_daily = [s.replace('-','') for s in btc_timestamps]

In [12]:
gold_dates_weekly = []
day_index = 0
for w in range(417):
    gold_dates_weekly.append(gold_dates_daily[day_index])
    day_index = (w+1)*7

In [13]:
# gold_dates_test = gold_dates_weekly[0:5]
# gold_dates_test
len(gold_dates_weekly)

417

In [23]:
# Extract Gold Price Data
# import http.client
# import mimetypes
# weekly_gold_price = []
# gold_24h_chg_pct = []
# conn=http.client.HTTPSConnection('www.goldapi.io')
# payload = ''
# headers = {'x-access-token':gold_api, 'Content_Type':'application/json'}
# for date in gold_dates_weekly:
#     conn.request('GET', f'/api/XAU/USD/{date}',payload,headers)
#     res= conn.getresponse()
#     gold_info = res.read()
#     gold_json_bytes = gold_info.decode('utf-8')
#     gold_json = json.loads(gold_json_bytes)
#     weekly_gold_price.append(gold_json['price'])
#     gold_24h_chg_pct.append(gold_json['chp'])

In [24]:
#combine weekly gold data with weekly crypto data
# cryptogold = {'Date':crypto_weekly_dates, 'BTC_Price':crypto_weekly_prices, 'Gold_Price':weekly_gold_price}
# crypto_v_gold = pd.DataFrame(cryptogold)
# crypto_v_gold

In [130]:
#crypto_v_gold.to_csv('Crypto_v_Gold.csv')

EXTRACT INFLATION DATA

In [15]:
# Query Bureau of Labor Statistics for inflation data
inflation_url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'
inf = requests.post(inflation_url, data = {'seriesid':'CUUR0000SA0', 'startyear':'2013', 'endyear':'2020',
                                           'catalog':'true', 'calculations':'true', 'aspects':'true', 'registrationkey':{bls_key}})
# inf_data = inf.json()
# inf_data
inf_data = inf.json()
inf_data

{'status': 'REQUEST_SUCCEEDED',
 'responseTime': 217,
 'message': ['No Aspect Available for Series CUUR0000SA0',
  'BLS does not produce net change calculations for Series CUUR0000SA0'],
 'Results': {'series': [{'seriesID': 'CUUR0000SA0',
    'catalog': {'series_title': 'All items in U.S. city average, all urban consumers, not seasonally adjusted',
     'series_id': 'CUUR0000SA0',
     'seasonality': 'Not Seasonally Adjusted',
     'survey_name': 'CPI for All Urban Consumers (CPI-U)',
     'survey_abbreviation': 'CU',
     'measure_data_type': 'All items',
     'area': 'U.S. city average',
     'item': 'All items'},
    'data': [{'year': '2020',
      'period': 'M12',
      'periodName': 'December',
      'value': '260.474',
      'aspects': [],
      'footnotes': [{}],
      'calculations': {'net_changes': {},
       'pct_changes': {'1': '0.1', '3': '0.1', '6': '1.0', '12': '1.4'}}},
     {'year': '2020',
      'period': 'M11',
      'periodName': 'November',
      'value': '260.229',

In [16]:
inf_months = []
inf_year = []
inf_value = []
inf_month_pct_chg = []
for i in range(95):
    inf_months.append(inf_data['Results']['series'][0]['data'][i]['period'])
    inf_year.append(inf_data['Results']['series'][0]['data'][i]['year'])
    inf_value.append(inf_data['Results']['series'][0]['data'][i]['value'])
    inf_month_pct_chg.append(inf_data['Results']['series'][0]['data'][i]['calculations']['pct_changes']['1'])
    

In [17]:
#Remove the 'M' in in the string for each month in inf_months
inf_months[:] = [s.replace('M','') for s in inf_months]

In [18]:
# Concatenate YYYY-MM
inf_year_month = [i + '-' + j for i,j in zip(inf_year, inf_months)] 

In [72]:
# Create lists of monthly Crypto data to merge with monthly inflation data
#inf_year_month
btc_monthly_prices = []
btc_month_end_index = -32
for i in range(8):
    for keyz, valuez in days_per_month.items():
        try:
            btc_monthly_prices.append(btc_prices[btc_month_end_index])
            btc_month_end_index = btc_month_end_index + int(valuez)
        except:
            print(f'no index at {valuez}')
    

In [74]:
# btc_monthly_prices.append(crypto_df['BTC Price'][2919])
# btc_monthly_prices.pop(0)

'28986.00603580'

In [75]:
# btc_monthly_prices.pop(0)

'33099.05748705'

In [86]:
# Reverse the order of the inflation data
ordered_inf_year_month = []
ordered_inf_value = []
ordered_inf_month_pct_chg = []
inf_months_index = 94
for e in inf_months:
        ordered_inf_year_month.append(inf_year_month[inf_months_index])
        ordered_inf_value.append(inf_value[inf_months_index])
        ordered_inf_month_pct_chg.append(inf_month_pct_chg[inf_months_index])
        inf_months_index = inf_months_index -1

In [101]:
#Calculate monthly change and monthly change pct for BTC prices
btc_monthly_change = [0]
btc_pct_monthly_change = [0]
btc_change_index = 1
for m in range(94):
        btc_monthly_change.append(float(btc_monthly_prices[btc_change_index])-float(btc_monthly_prices[btc_change_index-1]))
        btc_pct_monthly_change.append(((float(btc_monthly_prices[btc_change_index])-float(btc_monthly_prices[btc_change_index-1]))/
                                     float(btc_monthly_prices[btc_change_index-1])*100))
        btc_change_index=btc_change_index+1

In [102]:
# Merge crypto data with inflation data, create a dataframe, and crypto/inflation dataframe to CSV
crypto_inflation_data_dict = {'Date':ordered_inf_year_month, 'Inflation Value (1984=100)':ordered_inf_value,
                       'Monthly Change (%)':ordered_inf_month_pct_chg,'BTC_Price':btc_monthly_prices,'BTC_Pct_Change':btc_pct_monthly_change}
crypto_v_inflation = pd.DataFrame(crypto_inflation_data_dict)
crypto_v_inflation.to_csv('Crypto_v_Inflation.csv')

MERGE CRYPTO DATA WITH UNCERTAINTY DATA

In [111]:
# Open the uncertainty CSVs as dataframes
import os
twitter = os.path.join("Twitter_Uncertainty.csv")
twitter_uncertainty = pd.read_csv(twitter)
twitter_uncertainty['date'][1122]

'2016-02-28 00:00:00'

In [123]:
# Calculate raw daily change and percent daily change in btc prices
btc_daily_change = [0]
btc_pct_daily_change = [0]
btc_daily_change_index = 1
for m in range(2919):
        btc_daily_change.append(float(btc_prices[btc_daily_change_index])-float(btc_prices[btc_daily_change_index-1]))
        btc_pct_daily_change.append(((float(btc_prices[btc_daily_change_index])-float(btc_prices[btc_daily_change_index-1]))/
                                     float(btc_prices[btc_daily_change_index-1])*100))
        btc_daily_change_index=btc_daily_change_index+1

In [249]:
# Create df and export to csv full daily crypto prices with percent change and daily change calculations
crypto_prices_with_chg_dict = {'Date':btc_timestamps,'BTC_Price':btc_prices,'BTC_Change':btc_daily_change, 'BTC_pct_Change':btc_pct_daily_change}
crypto_prices_with_chg = pd.DataFrame(crypto_prices_with_chg_dict)
crypto_prices_with_chg.to_csv("Crypto_Prices_with_Change.csv")

In [127]:
# Create dataframe with dates, crypto prices, crypto changes, Twitter uncertainty
twitter_uncertainty_index = twitter_uncertainty['TEU']
short_crypto_dates = btc_timestamps[:2782]
short_crypto_prices = btc_prices[:2782]
short_btc_change = btc_daily_change[:2782]
short_btc_pct_change = btc_pct_daily_change[:2782]
crypto_v_twitter_dict = {'Date':short_crypto_dates, 'BTC_Price':short_crypto_prices,'BTC_Change':short_btc_change,
                         'BTC_pct_Change':short_btc_pct_change,'TEU':twitter_uncertainty_index}
crypto_v_twitter = pd.DataFrame(crypto_v_twitter_dict)
crypto_v_twitter.to_csv('Crypto_v_Twitter.csv')

In [141]:
# Open Categorical Uncertainty Data
unc_path = os.path.join("Categorical_EPU_Data.csv")
cat_uncertainty = pd.read_csv(unc_path)

In [143]:
cat_uncertainty=cat_uncertainty.drop(index=0)

In [144]:
cat_uncertainty['BTC_Price'] = btc_monthly_prices
cat_uncertainty['BTC_Pct_Change'] = btc_pct_monthly_change
cat_uncertainty.to_csv('Crypto_v_Uncertainty.csv')

In [235]:
golden_road = os.path.join("Crypto_v_Gold.csv")
gooollld = pd.read_csv(golden_road)

In [236]:
gold_weekly_prices = gooollld['Gold_Price']
crypto_gold_datez = gooollld['Date']

In [237]:
#Calculate btc weekly change, btc weekly change pct, gold weekly change, gold weekly change pct
btc_wkly_change = [0]
btc_pct_wkly_change = [0]
gold_wkly_change = [0]
gold_pct_wkly_change = [0]
gold_wkly_change_index = 1
for m in range(416):
        btc_wkly_change.append(float(crypto_weekly_prices[gold_wkly_change_index])-float(crypto_weekly_prices[gold_wkly_change_index-1]))
        btc_pct_wkly_change.append(((float(crypto_weekly_prices[gold_wkly_change_index])-float(crypto_weekly_prices[gold_wkly_change_index-1]))/
                                     float(crypto_weekly_prices[gold_wkly_change_index-1])*100))
        gold_wkly_change.append(float(gold_weekly_prices[gold_wkly_change_index])-float(gold_weekly_prices[gold_wkly_change_index-1]))
        gold_pct_wkly_change.append(((float(gold_weekly_prices[gold_wkly_change_index])-float(gold_weekly_prices[gold_wkly_change_index-1]))/
                                     float(gold_weekly_prices[gold_wkly_change_index-1])*100))
        gold_wkly_change_index=gold_wkly_change_index+1

In [245]:
gooollld['BTC_Change']=btc_wkly_change
gooollld['BTC_pct_Change'] = btc_pct_wkly_change
gooollld['Gold_Change'] = gold_wkly_change
gooollld['Gold_pct_Change'] = gold_pct_wkly_change
gooollld.to_csv("btc_v_gold.csv")
# btc_v_gold_dict = {'Date':crypto_gold_datez,'BTC_Price':}

MERGE STOCK DATA WITH CRYPTO DATA

In [239]:
stock_path = os.path.join("Stock_Market_Data.csv")
stock_data = pd.read_csv(stock_path)
crypto_v_stockmarket = stock_data.merge(crypto_v_twitter, how='inner', on='Date')
crypto_v_stockmarket = crypto_v_stockmarket.drop(columns='TEU')
crypto_v_stockmarket.to_csv("crypto_v_stockmarket.csv")

In [240]:
cat_uncertainty

,Date,Economic Policy Uncertainty,Monetary policy,Government spending,National security,Financial Regulation,"Sovereign debt, currency crises",BTC_Price,BTC_Pct_Change
1,3/1/2013,127.907071,74.678109,261.072394,67.232097,76.095169,243.156035,33.85347654,0.000000
2,4/1/2013,77.201711,34.016665,72.652522,38.559536,47.961099,28.380715,93.79780464,177.069933
3,5/1/2013,71.645563,35.081088,53.429494,31.160853,37.286633,27.580198,139.88375302,49.133291
4,6/1/2013,77.763068,55.337061,57.108591,36.726217,89.548832,38.213961,127.94651362,-8.533685
5,7/1/2013,86.720342,51.459549,71.349203,91.145834,153.359947,15.468733,89.73516147,-29.865098
...,...,...,...,...,...,...,...,...,...
91,9/1/2020,225.148692,124.028643,97.974717,137.845947,102.927451,137.040186,11670.62656357,2.793402
92,10/1/2020,255.857027,108.102130,119.907785,95.426135,136.676648,75.822797,10784.41101995,-7.593556
93,11/1/2020,252.119123,146.897536,147.805234,169.058055,101.620142,71.210317,13893.46573199,28.829156
94,12/1/2020,224.380454,80.474777,154.953537,70.908410,36.542788,34.752845,19531.16867786,40.578089


In [241]:
crypto_v_twitter

,Date,BTC_Price,BTC_Change,BTC_pct_Change,TEU
0,2013-02-01,20.56073352,0.000000,0.000000,101.457894
1,2013-02-02,19.98875651,-0.571977,-2.781890,26.352700
2,2013-02-03,20.74686364,0.758107,3.792668,27.670335
3,2013-02-04,20.52266187,-0.224202,-1.080654,80.375734
4,2013-02-05,20.76371547,0.241054,1.174573,94.869719
...,...,...,...,...,...
2777,2020-09-11,10422.24255140,14.065334,0.135137,260.891728
2778,2020-09-12,10467.73825792,45.495707,0.436525,150.210389
2779,2020-09-13,10367.46538855,-100.272869,-0.957923,142.304579
2780,2020-09-14,10688.95501140,321.489623,3.100947,304.373683


In [242]:
crypto_v_inflation

,Date,Inflation Value (1984=100),Monthly Change (%),BTC_Price,BTC_Pct_Change
0,2013-02,232.166,0.8,33.85347654,0.000000
1,2013-03,232.773,0.3,93.79780464,177.069933
2,2013-04,232.531,-0.1,139.88375302,49.133291
3,2013-05,232.945,0.2,127.94651362,-8.533685
4,2013-06,233.504,0.2,89.73516147,-29.865098
...,...,...,...,...,...
90,2020-08,259.918,0.3,11670.62656357,2.793402
91,2020-09,260.280,0.1,10784.41101995,-7.593556
92,2020-10,260.388,0.0,13893.46573199,28.829156
93,2020-11,260.229,-0.1,19531.16867786,40.578089


In [243]:
gooollld

,Date,BTC_Price,Gold_Price,BTC_Change,BTC_pct_Change,Gold_Change,Gold_pct_Change
0,2/1/2013,20.560734,1665.00,0.000000,0.000000,0.00,0.000000
1,2/8/2013,22.888735,1669.75,2.328001,11.322559,4.75,0.285285
2,2/15/2013,27.502657,1629.25,4.613922,20.158048,-40.50,-2.425513
3,2/22/2013,30.706054,1580.00,3.203398,11.647593,-49.25,-3.022863
4,3/1/2013,34.944449,1570.00,4.238395,13.803124,-10.00,-0.632911
...,...,...,...,...,...,...,...
412,12/27/2020,26310.707720,1872.55,3220.537366,13.657307,-6.40,-0.340616
413,1/3/2021,33117.849620,1891.10,5235.270774,19.533431,18.55,0.990628
414,1/10/2021,38363.108450,1891.30,5559.165851,17.352403,0.20,0.010576
415,1/17/2021,35829.897020,1853.85,-1774.059251,-4.718741,-37.45,-1.980119


In [244]:
crypto_v_stockmarket

,Date,DJIA_Close,DJIA_Change,DJIA_pct_Change,Close,NDAQ_Change,NDAQ_pct_Change,BTC_Price,BTC_Change,BTC_pct_Change
0,2013-02-01,14009.79,0.00,0.000000,3179.10,0.00,0.000000,20.56073352,0.000000,0.000000
1,2013-02-04,13880.08,-129.71,-0.925853,3131.17,-47.93,-0.015077,20.52266187,-0.224202,-1.080654
2,2013-02-05,13979.30,99.22,0.714837,3171.58,40.41,0.012906,20.76371547,0.241054,1.174573
3,2013-02-06,13986.52,7.22,0.051648,3168.48,-3.10,-0.000977,21.27228065,0.508565,2.449298
4,2013-02-07,13944.05,-42.47,-0.303650,3165.13,-3.35,-0.001057,22.14472461,0.872444,4.101318
...,...,...,...,...,...,...,...,...,...,...
1913,2020-09-09,27940.47,439.58,1.598421,11141.56,293.87,0.027091,10264.65200779,69.980012,0.686437
1914,2020-09-10,27534.58,-405.89,-1.452696,10919.59,-221.97,-0.019923,10408.17721691,143.525209,1.398247
1915,2020-09-11,27665.64,131.06,0.475983,10853.55,-66.04,-0.006048,10422.24255140,14.065334,0.135137
1916,2020-09-14,27993.33,327.69,1.184466,11056.65,203.10,0.018713,10688.95501140,321.489623,3.100947
